In [ ]:
import re
import os
import boto3
import pandas as pd
from tqdm import tqdm
from decimal import Decimal
from zipfile import ZipFile
from urllib.request import urlretrieve

In [ ]:
# urlretrieve(
#     "http://files.grouplens.org/datasets/movielens/ml-1m.zip", "movielens.zip")
# ZipFile("movielens.zip", "r").extractall()

In [ ]:
movies = pd.read_csv(
    "ml-1m/movies.dat",
    sep="::",
    names=["movie_id", "title", "genres"],
    engine="python",
    encoding="ISO-8859-1",
)

In [ ]:
ratings = pd.read_csv(
    "ml-1m/ratings.dat",
    sep="::",
    names=["user_id", "movie_id", "rating", "unix_timestamp"],
    engine="python",
)

In [ ]:
ratings = ratings.groupby(['movie_id'])[
    'rating'].mean().reset_index(name="rating")

In [ ]:
def round_to_half(x):
    lower = int(x)
    middle = lower + 0.5
    upper = lower+1

    if x >= middle:
        if abs(x-upper) > abs(x-middle):
            return middle
        else:
            return upper

    if x < middle:
        if abs(x-lower) > abs(x-middle):
            return middle
        else:
            return lower

In [ ]:
# ratings['rating'] = ratings['rating'].apply(lambda x: round_to_half(x))
ratings['rating'] = ratings['rating'].apply(lambda x: round(x, 1))

In [ ]:
movies = movies.merge(ratings)

In [ ]:
movies.head()

In [ ]:
def extract_year(movie_title):
    # Regular expression pattern to match the year in parentheses
    pattern = r'\((\d{4})\)'
    match = re.search(pattern, movie_title)
    if match:
        return match.group(1)
    else:
        return None

In [ ]:
def remove_year(movie_title):
    # Regular expression pattern to match the year in parentheses
    pattern = r'\((\d{4})\)'
    movie_title = re.sub(pattern, "", movie_title)
    return movie_title

In [ ]:
movies['release_year'] = movies['title'].apply(
    lambda x: extract_year(x)).astype("int")

In [ ]:

movies['title'] = movies['title'].apply(lambda x: remove_year(x).strip())

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))

In [ ]:
movies.head()

In [ ]:
def list_s3_files(bucket_name, prefix=''):
    s3 = boto3.client('s3')
    files = []
    continuation_token = None

    while True:
        if continuation_token:
            response = s3.list_objects_v2(
                Bucket=bucket_name, Prefix=prefix, ContinuationToken=continuation_token)
        else:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

        for obj in response['Contents']:
            files.append(obj['Key'])

        if 'NextContinuationToken' in response:
            continuation_token = response['NextContinuationToken']
        else:
            break

    return files

In [ ]:
file_paths = list_s3_files("cloudfront-aws-bucket",
                           prefix='ml-1m-cover-images')

In [ ]:
df_images = pd.DataFrame()

In [ ]:
df_images['image_path'] = file_paths

In [ ]:
df_images = df_images.iloc[1:].copy()

In [ ]:
df_images['movie_id'] = df_images['image_path'].apply(
    lambda x: os.path.basename(os.path.dirname(x))).astype('int')

In [ ]:
df_images['image_url'] = df_images['image_path'].apply(
    lambda x: os.path.join("http://d2gewc5xha837s.cloudfront.net", x))

In [ ]:
df_images = df_images.drop("image_path", axis=1)

In [ ]:
movies = movies.merge(df_images, on=['movie_id'])

In [ ]:
movies.head()

In [ ]:
movies.to_parquet("./movie_info.parquet")

In [ ]:
movies_genres = movies.explode(['genres'])

In [ ]:
movies_genres = movies_genres.rename(columns={'genres': "genre"})

In [ ]:
movies_genres.head()

In [ ]:
movies_genres['rank'] = movies_genres.groupby('movie_id')['genre'].rank()

In [ ]:
movies_genres['rank'] = movies_genres['rank'].astype("int")

In [ ]:
movies_genres.head()

In [ ]:
movies_genres = movies_genres.merge(
    movies[['movie_id', 'genres']], on=['movie_id'])

In [ ]:
movies_genres.head()

In [ ]:
genres = list(set(movies_genres['genre']))

In [ ]:
dynamodb = boto3.resource("dynamodb")

In [ ]:
table_name = "movielens_movie"

In [ ]:
table = dynamodb.Table(table_name)

In [ ]:
items = []
for i in range(len(movies_genres)):
    item = movies_genres.iloc[i].to_dict()
    item["rating"] = Decimal(str(item["rating"]))
    items.append(item)

In [ ]:
with table.batch_writer() as writer:
    for item in tqdm(items):
        writer.put_item(Item=item)